In [1]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd 

import collections
import os

## Load data


In [2]:
#file = '/home/parallels/Desktop/Parallels Shared Folders/Home/Desktop/CCSI2/RPB_sensitivity_8_25_22.xlsx'
file = '/Users/wangjialu/Desktop/CCSI2/RPB_sensitivity_8_25_22.xlsx'

data = pd.ExcelFile(file)

In [3]:
df1 = pd.read_excel(data, 'MTC')
df2 = pd.read_excel(data, 'HTC')
df3 = pd.read_excel(data, 'DH')
df4 = pd.read_excel(data, 'Iso_1')
df5 = pd.read_excel(data, 'Iso_2')

In [4]:
col_names_up = df1.columns.values[4:18].tolist()
col_names_lo = df1.columns.values[22:].tolist()
print('===Measurement names===')
print(col_names_up)
print(col_names_lo)

===Measurement names===
['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
['Ads.gas_inlet.F.1', 'Ads.gas_outlet.F.1', 'Ads.gas_outlet.T.1', 'Ads.gas_outlet.z("CO2").1', 'Des.gas_inlet.F.1', 'Des.gas_outlet.F.1', 'Des.gas_outlet.T.1', 'Des.gas_outlet.z("CO2").1', 'Ads.T_g.Value(19,10).1', 'Ads.T_g.Value(23,10).1', 'Ads.T_g.Value(28,10).1', 'Ads.z("CO2",19,10).1', 'Ads.z("CO2",23,10).1', 'Ads.z("CO2",28,10).1']


In [5]:
timeSet = list(df1['Time'][1:].tolist())
print(timeSet[0], timeSet[-1])
print(len(timeSet))

0 220
2201


In [6]:
# 10 scenarios in total. We are going to map them from 0 to 9 with
# MTC_up, MTC_lo, HTC_up, HTC_lo, DH_up, DH_lo, LSO1_up, LSO1_do, LSO1_up, LSO1_do

# output record: key: 1-10 scenario numbers; Value: a list of 14 lists, each list contains the data of one measurement
output_record = {}
# output_doe: key: 1-10 scenario numbers; Value: a large list containing all data of measurements
output_doe = {}

for s in [0,2,4,6,8]:
    
    df = [df1, df2, df3, df4, df5][int(s/2)]
    
    output_up = []
    output_all_up = []
    
    for nam in col_names_up:
        output_up.append(df[nam].values[1:].tolist())
        
        for idx in list(df[nam].values[1:].tolist()):
            output_all_up.append(idx)
        
    output_lo = []
    output_all_lo = []
    
    for nam in col_names_lo:
        output_lo.append(df[nam].values[1:].tolist())
        
        for idx in list(df[nam].values[1:].tolist()):
            output_all_lo.append(idx)
        
        
    output_record[s] = output_up
    output_record[s+1] = output_lo
    output_doe[s] = output_all_up
    output_doe[s+1] = output_all_lo

In [7]:
output = pd.DataFrame(output_record)
#output.to_csv('RotaryBed.csv')

### Scale 

Scale with y = (y-minimum)/(max_y-minimum)

In [8]:
output_record_scale = collections.defaultdict(list)
output_doe_scale = {}

for i in range(10):
    for idx in range(14):
        col = np.asarray(output_record[i][idx])
        max_col, min_col = max(col), min(col)
        col = (col-min_col)/(max_col-min_col)
        output_record_scale[i].append(col)
        
    output_doe_scale[i] = []
    for idx2 in range(14):
        output_doe_scale[i] = np.concatenate([output_doe_scale[i], output_record_scale[i][idx2]])

In [9]:
output_record_stand = collections.defaultdict(list)
output_doe_stand = {}

for i in range(10):
    for idx in range(14):
        col = np.asarray(output_record[i][idx])
        avg_col = sum(col)/len(col)
        col = (col-avg_col)/avg_col
        output_record_stand[i].append(col)
        
    output_doe_stand[i] = []
    for idx2 in range(14):
        output_doe_stand[i] = np.concatenate([output_doe_stand[i], output_record_stand[i][idx2]])

### draw figures

In [ ]:
def plot_data(output, scena, measure_number, measure_label, para_label1, para_label2):
    sense = []

    plt.plot(timeSet, output[scena][measure_number], linewidth=2, color='red', label=para_label1)
    plt.plot(timeSet, output[scena+1][measure_number], linewidth=2, color='blue', label=para_label2)
    plt.xlabel('Time (min)')
    plt.ylabel(measure_label)
    plt.legend()
    plt.show()



In [ ]:
#print(df1.iloc[0])
col_names_unit = [
                  'Ads.gas_inlet.F (kmol/hr)', 
                  'Ads.gas_outlet.F (kmol/hr)', 
                  'Ads.gas_outlet.T (C)', 
                  'Ads.gas_outlet.z("CO2") (kmol/kmol)', 
                  'Des.gas_inlet.F (kmol/hr)', 
                  'Des.gas_outlet.F (kmol/hr)', 
                  'Des.gas_outlet.T (C)', 
                  'Des.gas_outlet.z("CO2") (kmol/kmol)', 
                  'Ads.T_g.Value(19,10) (C)', 
                  'Ads.T_g.Value(23,10) (C)', 
                  'Ads.T_g.Value(28,10) (C)', 
                  'Ads.z("CO2",19,10) (kmol/kmol)', 
                  'Ads.z("CO2",23,10) (kmol/kmol)', 
                  'Ads.z("CO2",28,10) (kmol/kmol)']

In [ ]:
for me, mename in enumerate(col_names_unit):

    plot_data(output_record_stand, 0, me, mename, 'MTC+3%', 'MTC-3%')
    plot_data(output_record_stand, 2, me, mename, 'HTC+3%', 'HTC-3%')
    plot_data(output_record_stand, 4, me, mename, 'DH+3%', 'DH-3%')
    plot_data(output_record_stand, 6, me, mename, 'Iso1+1%', 'Iso1-1%')
    plot_data(output_record_stand, 8, me, mename, 'Iso2+1%', 'Iso2-1%')

### DoE to get Q matrix

In [24]:
# total measurements 
num_measure = 14 
num_time = 2201
num_overall = num_measure*num_time
num_para = 5
print(num_overall)

30814


In [25]:
para_set = ['MTC', 'HTC', 'DH', 'ISO1', 'ISO2']
def FIM_Compute(output, output_region):
    # overall Q
    Q = np.zeros((num_overall,num_para))
    # Q as a set. Q[key]: [meausre1, ..., meausre14].
    Q_region = collections.defaultdict(list)

    for i in [0,2,4,6,8]:
        # first 3 parameters
        if i in [0,2,4]:
            for k in range(num_measure):
                measure1 = []
                for n in range(num_time):
                    measure1.append((output_region[i][k][n]-output_region[i+1][k][n])/0.06)
                Q_region[para_set[int(i/2)]].append(measure1)
            
            for j in range(num_overall):
                Q[j][int(i/2)] = (output[i][j]-output[i+1][j])/0.06
                
        else:
            for k in range(num_measure):
                measure1 = []
                for n in range(num_time):
                    measure1.append((output_region[i][k][n]-output_region[i+1][k][n])/0.02)
                Q_region[para_set[int(i/2)]].append(measure1)

            for l in range(num_overall):
                Q[l][int(i/2)] = (output[i][l]-output[i+1][l])/0.02
                
    FIM = Q.T@Q
    
    return Q, Q_region, FIM
        

In [11]:
Q_overall, Q_overall_region, FIM_sat = FIM_Compute(output_doe_scale, output_record_scale)

print(FIM_sat)

print('Determinant:', np.linalg.det(FIM_sat))
print('Eigenvalue:', np.linalg.eigvals(FIM_sat))

[[ 1.12815147e+05  1.11889258e+02 -8.27430498e+04  2.21615060e+05
  -1.29841767e+04]
 [ 1.11889258e+02  1.26962354e+02  2.53888465e+01  1.47809182e+03
  -3.94892943e+02]
 [-8.27430498e+04  2.53888465e+01  6.67741948e+04 -1.71132603e+05
   3.53086303e+03]
 [ 2.21615060e+05  1.47809182e+03 -1.71132603e+05  5.10219001e+05
  -2.22797772e+04]
 [-1.29841767e+04 -3.94892943e+02  3.53086303e+03 -2.22797772e+04
   1.19156218e+04]]
Determinant: 1.5339669750178904e+19
Eigenvalue: [6.70097327e+05 1.69330949e+04 1.35877177e+04 1.14596606e+03
 8.68207456e+01]


In [14]:
print(len(Q_overall_region['HTC'][2]))

[0.82924264 0.82889201 0.82889201 ... 0.24395832 0.24368765 0.24343632]
[0.81784261 0.81784261 0.81784261 ... 0.23950935 0.23919931 0.23888927]
2201


### Data export

In [15]:
Q_set = {}
para_set = ['MTC', 'HTC', 'DH', 'ISO1', 'ISO2']

for i in range(num_para):
    Q_set[para_set[i]] = Q_overall[:, i]

In [16]:
Q_dict = pd.DataFrame(Q_set)
Q_dict.to_csv('Q_overall.csv')

In [17]:
# Q_set10 accepts 220 timepoints/measurement, equally distributed. In total there should be 220*14 = 3080 
Q_set10 = collections.defaultdict(list)

time_set_overall = []
for i in range(14):
    
    time_set = np.linspace(2200*i+10+i, 2200*(i+1)+i, 220)
    for k in range(len(time_set)):
        time_set_overall.append(time_set[k]) 
print(time_set_overall)

for i in range(5):
    for k in time_set_overall:
        Q_set10[para_set[i]].append(Q_overall[int(k)][i])
    
print(len(Q_set10['MTC']))

[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 110.0, 120.0, 130.0, 140.0, 150.0, 160.0, 170.0, 180.0, 190.0, 200.0, 210.0, 220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 290.0, 300.0, 310.0, 320.0, 330.0, 340.0, 350.0, 360.0, 370.0, 380.0, 390.0, 400.0, 410.0, 420.0, 430.0, 440.0, 450.0, 460.0, 470.0, 480.0, 490.0, 500.0, 510.0, 520.0, 530.0, 540.0, 550.0, 560.0, 570.0, 580.0, 590.0, 600.0, 610.0, 620.0, 630.0, 640.0, 650.0, 660.0, 670.0, 680.0, 690.0, 700.0, 710.0, 720.0, 730.0, 740.0, 750.0, 760.0, 770.0, 780.0, 790.0, 800.0, 810.0, 820.0, 830.0, 840.0, 850.0, 860.0, 870.0, 880.0, 890.0, 900.0, 910.0, 920.0, 930.0, 940.0, 950.0, 960.0, 970.0, 980.0, 990.0, 1000.0, 1010.0, 1020.0, 1030.0, 1040.0, 1050.0, 1060.0, 1070.0, 1080.0, 1090.0, 1100.0, 1110.0, 1120.0, 1130.0, 1140.0, 1150.0, 1160.0, 1170.0, 1180.0, 1190.0, 1200.0, 1210.0, 1220.0, 1230.0, 1240.0, 1250.0, 1260.0, 1270.0, 1280.0, 1290.0, 1300.0, 1310.0, 1320.0, 1330.0, 1340.0, 1350.0, 1360.0, 1370.0, 1380.0, 139

In [18]:
Q_dict10 = pd.DataFrame(Q_set10)
print(Q_dict10)
Q_dict10.to_csv('Q10.csv')

           MTC       HTC        DH      ISO1      ISO2
0    -0.540656 -0.002425 -0.049803  1.898348  0.552470
1    -0.534657 -0.002425 -0.049803  1.916263  0.552470
2    -0.540147 -0.002339 -0.055566  1.916263  0.552470
3    -0.540147 -0.002339 -0.055566  1.916263  0.569431
4    -0.540147 -0.002339 -0.049833  1.916263  0.551899
...        ...       ...       ...       ...       ...
3075 -0.199989  0.033213 -0.015159  0.892123  0.107665
3076 -0.224126  0.032914 -0.020892  0.891896  0.141038
3077 -0.246716  0.032423 -0.025973  0.891157  0.171950
3078 -0.267915  0.031736 -0.030723  0.889418  0.200883
3079 -0.287246  0.031342 -0.034661  0.886191  0.227353

[3080 rows x 5 columns]


In [19]:
# Q_set10 accepts 220 timepoints/measurement, equally distributed. In total there should be 220*14 = 3080 
Q_set100 = {}

time_set100 = []
for i in range(14):
    time_set = np.linspace(2200*i+100+i, 2200*(i+1)+i, 22)
    for k in range(len(time_set)):
        time_set100.append(time_set[k])
print(time_set100)

for i in range(5):
    
    Q_set100[para_set[i]] = [Q_overall[int(k),i] for k in time_set100]
    
print(len(Q_set100['MTC']))



[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0, 1500.0, 1600.0, 1700.0, 1800.0, 1900.0, 2000.0, 2100.0, 2200.0, 2301.0, 2401.0, 2501.0, 2601.0, 2701.0, 2801.0, 2901.0, 3001.0, 3101.0, 3201.0, 3301.0, 3401.0, 3501.0, 3601.0, 3701.0, 3801.0, 3901.0, 4001.0, 4101.0, 4201.0, 4301.0, 4401.0, 4502.0, 4602.0, 4702.0, 4802.0, 4902.0, 5002.0, 5102.0, 5202.0, 5302.0, 5402.0, 5502.0, 5602.0, 5702.0, 5802.0, 5902.0, 6002.0, 6102.0, 6202.0, 6302.0, 6402.0, 6502.0, 6602.0, 6703.0, 6803.0, 6903.0, 7003.0, 7103.0, 7203.0, 7303.0, 7403.0, 7503.0, 7603.0, 7703.0, 7803.0, 7903.0, 8003.0, 8103.0, 8203.0, 8303.0, 8403.0, 8503.0, 8603.0, 8703.0, 8803.0, 8904.0, 9004.0, 9104.0, 9204.0, 9304.0, 9404.0, 9504.0, 9604.0, 9704.0, 9804.0, 9904.0, 10004.0, 10104.0, 10204.0, 10304.0, 10404.0, 10504.0, 10604.0, 10704.0, 10804.0, 10904.0, 11004.0, 11105.0, 11205.0, 11305.0, 11405.0, 11505.0, 11605.0, 11705.0, 11805.0, 11905.0, 12005.0, 12105.0, 12205.0, 12305.0, 

In [20]:
Q_dict100 = pd.DataFrame(Q_set100)
Q_dict100.to_csv('Q100.csv')

### Data export with Q_region

In [27]:
Q_set10 = collections.defaultdict(list)

time_set = np.linspace(10, 2200, 220)
print(time_set)

for i in range(num_para):
    for k in range(num_measure):
        Q_set10[para_set[i]].append([Q_overall_region[para_set[i]][k][int(n)] for n in time_set])
        
print(Q_set10)

[  10.   20.   30.   40.   50.   60.   70.   80.   90.  100.  110.  120.
  130.  140.  150.  160.  170.  180.  190.  200.  210.  220.  230.  240.
  250.  260.  270.  280.  290.  300.  310.  320.  330.  340.  350.  360.
  370.  380.  390.  400.  410.  420.  430.  440.  450.  460.  470.  480.
  490.  500.  510.  520.  530.  540.  550.  560.  570.  580.  590.  600.
  610.  620.  630.  640.  650.  660.  670.  680.  690.  700.  710.  720.
  730.  740.  750.  760.  770.  780.  790.  800.  810.  820.  830.  840.
  850.  860.  870.  880.  890.  900.  910.  920.  930.  940.  950.  960.
  970.  980.  990. 1000. 1010. 1020. 1030. 1040. 1050. 1060. 1070. 1080.
 1090. 1100. 1110. 1120. 1130. 1140. 1150. 1160. 1170. 1180. 1190. 1200.
 1210. 1220. 1230. 1240. 1250. 1260. 1270. 1280. 1290. 1300. 1310. 1320.
 1330. 1340. 1350. 1360. 1370. 1380. 1390. 1400. 1410. 1420. 1430. 1440.
 1450. 1460. 1470. 1480. 1490. 1500. 1510. 1520. 1530. 1540. 1550. 1560.
 1570. 1580. 1590. 1600. 1610. 1620. 1630. 1640. 16

In [28]:
Q_dict10 = pd.DataFrame(Q_set10)
print(Q_dict10)
Q_dict10.to_csv('Q10_test.csv')

                                                  MTC  \
0   [-0.5406563522351582, -0.5346568321965617, -0....   
1   [0.07103975678984242, 0.0687214000410293, 0.06...   
2   [-0.290510962076683, -0.290510962076683, -0.29...   
3   [-0.2193254645776748, -0.2193519789458101, -0....   
4   [-0.13702347035614756, -0.13702347035614756, -...   
5   [0.3137689184186813, 0.3137689184186813, 0.313...   
6   [7.11955077086688, 7.11955077086688, 7.1195507...   
7   [0.5884231927850286, 0.5884231927850286, 0.583...   
8   [0.08957254639560773, 0.08957254639560773, 0.0...   
9   [1.0380847022964912, 1.0380847022964912, 1.038...   
10  [-0.6932468662462684, -0.6955061405673351, -0....   
11  [-0.09544361112974557, -0.09544361112974557, -...   
12  [-0.08334114690796794, -0.08334114690796794, -...   
13  [-0.23695897085486234, -0.23695897085486234, -...   

                                                  HTC  \
0   [-0.002424671233954475, -0.002424671233954475,...   
1   [0.004546862712410486, 0.0

### Test

In [23]:
print(Q_overall[28713])
print(Q_overall_region['DH'][13][10])
print(Q_set['DH'][28713])
print(Q_set10['DH'][2860])
print(Q_set100['DH'][286])

[-0.23695212 -0.01605029  0.01468072 -0.17635062  0.29497461]
0.014680720979787232
0.014680720979787232
0.014680720979787232
0.014680720979787232
